In [ ]:
# default_exp core

# Cleanup
> This notebook is provided to help you clean up any resources you have created by running through the example. You should also go to the [CloudFormation console](https://console.aws.amazon.com/cloudformation/home) and delete the stack that you created.

In [ ]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import json
import sys

role = get_execution_role()
sm = boto3.Session().client(service_name='sagemaker')

### Delete feature groups

In [ ]:
%store -r
try:
    sm.delete_feature_group(FeatureGroupName=fg_name) 
    print('deleted '+fg_name)
except:
    pass

deleted heidelberg-mlops-fg


In [ ]:
sm.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'transaction-feature-group-15-05-22-59',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:082830052325:feature-group/transaction-feature-group-15-05-22-59',
   'CreationTime': datetime.datetime(2020, 12, 15, 5, 23, 10, 113000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'ml-ops-2',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:082830052325:feature-group/ml-ops-2',
   'CreationTime': datetime.datetime(2020, 12, 19, 6, 49, 19, 335000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'ml-ops-1',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:082830052325:feature-group/ml-ops-1',
   'CreationTime': datetime.datetime(2020, 12, 18, 7, 7, 15, 38000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'identity-feature-group-15-05-22-59',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:0828300523

### Stop the KDA SQL App

In [ ]:
%store -r
import boto3
kda_client = boto3.client('kinesisanalytics')

try:
    kda_client.stop_application(ApplicationName=APPLICATION_NAME)
except:
    pass

print('Stopped the KDA SQL app')

Stopped the KDA SQL app


### Delete the KDA SQL App

In [ ]:
import time
%store -r 

try:
    ready = False
    while not ready:
        app_desc = kda_client.describe_application(ApplicationName=APPLICATION_NAME)['ApplicationDetail']
        if app_desc['ApplicationStatus'] == 'READY':
            ready = True
        else:
            print('Waiting for KDA SQL app to be ready for deletion...')
            time.sleep(15)
    create_timestamp = app_desc['CreateTimestamp']
    response = kda_client.delete_application(ApplicationName=APPLICATION_NAME,
                                  CreateTimestamp=create_timestamp)
    print('Deleted KDA SQL app')
except:
    print('FAILED to delete KDA sql app')

Waiting for KDA SQL app to be ready for deletion...
Deleted KDA SQL app


### Remove the trigger from Lambda

In [ ]:
import boto3

%store -r 

lambda_client = boto3.client('lambda')
paginator = lambda_client.get_paginator('list_event_source_mappings')
mapping_iterator = paginator.paginate(FunctionName=lambda_to_model_arn)

for m in mapping_iterator:
    if len(m['EventSourceMappings']) > 0:
        uuid = m['EventSourceMappings'][0]['UUID']
        print(f'Deleting mapping: {uuid}...')
        lambda_client.delete_event_source_mapping(UUID=uuid)

Deleting mapping: 9974f2db-2d9a-4081-9e31-7cbb227ca548...


### Delete the Kinesis data stream

In [ ]:
%store -r
kinesis_client = boto3.client('kinesis')
try:
    kinesis_client.delete_stream(StreamName=STREAM_NAME)
except:
    pass
print('deleted Kinesis stream')

deleted Kinesis stream


### Delete the SageMaker endpoint

In [ ]:
%store -r
#endpoint_name="sagemaker-xgboost-2020-12-23-06-52-57-726"
try:
    sm.delete_endpoint(EndpointName=endpoint_name)
except:
    print("yes")
    pass